Libraries and options

In [2]:
import numpy as np, pandas as pd, sys, os, re, zipfile, shutil, pickle, matplotlib.pyplot as plt, seaborn as sns, bz2
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pd.options.display.max_colwidth = 50

Importing census data

In [ ]:
censusDF = pd.read_csv('./censusData.csv')
varDF = pd.read_csv("./acs2019Variables.csv", skiprows=[1,2,3])
pd.options.display.max_colwidth=1000
#change state and county fip codes to strings with standard lengths
censusDF['state'] = censusDF['state'].astype(str).str.zfill(2)
censusDF['county'] = censusDF['county'].astype(str).str.zfill(3)
#join these to create the combined fips code
censusDF['fips'] = censusDF['state']+censusDF['county']
#create a list of descriptive variable names in the same order as in censusDF
a = []
for i in censusDF.columns[:-4]:
    vName = varDF['label'][varDF['name']==i].to_string(index=False)+varDF['concept'][varDF['name']==i].to_string(index=False)
    a.append(vName)
for i in censusDF.columns[-4:]:
    a.append(i)
#rebuild dataframe with these column names
# censusDF = pd.DataFrame(censusDF.values, columns=a)
#et voila

Importing covid data

In [ ]:
path = r'../covid/processed_data/county_merged_parts/'
listFiles = []
for r, d, files in os.walk(path):
    for file in files:
        listFiles.append(file)

testDF = pd.DataFrame()
for i,file in enumerate(listFiles):
    inDF = pd.read_pickle(f'{path}{file}')
    inDF.reset_index(level=0,inplace=True)
    testDF = pd.concat([testDF,inDF],ignore_index=True)
# test = f'{path}{listFiles[0]}'
# testDF= pd.read_pickle(test)

# mask = testDF['fips'].isnull()
testDF.dropna(subset=['fips'], inplace=True)

testDF['fips']=testDF['fips'].astype(int).astype(str)

testDF['fips'] = testDF['fips'].str.zfill(5)

Joining census and covid data

In [ ]:
newDF = testDF.join(censusDF.set_index('fips'),on='fips',how='left')

Exported this DF for future work

In [ ]:
# newDF.to_csv('covidCensus.gz',index=False,compression='gzip')

Imported back

In [ ]:
anotherDF = pd.read_csv('covidCensus.gz')
anotherDF['fips'] = anotherDF['fips'].astype(str)
anotherDF['dates']=pd.to_datetime(anotherDF['dates'])

Import air quality data

In [ ]:
path = r'../air_quality/'
dirs = next(os.walk(path))[1]
airDF = pd.DataFrame()
cols = []
for dir in dirs:
    files = next(os.walk(f'{path}{dir}/unzipped/'))[2]
    for m,file in enumerate(files):
        inDF = pd.read_csv(f'{path}{dir}/unzipped/{file}')
        cols.append(inDF.columns.values)
        inDF['Date of Last Change'] = pd.to_datetime(inDF['Date of Last Change'])
        inDF['fips'] = inDF['fips'].astype(str).str.zfill(5)
        prefix = re.search('(?<=daily_)[^_]*(?=_)', f'{file}').group(0)
        print(prefix)
        inDF['pollutant'] = prefix
#         inDF = inDF.add_prefix(prefix+'_')
        airDF =pd.concat([airDF,inDF],ignore_index=True)


In [ ]:
airDF.columns

In [ ]:
for m,col in enumerate(cols[1:]):
    try:
        print(m+1,all(col==cols[0]),"col0")
    except:
        print(m+1,all(col==cols[3]),"col3")


In [ ]:
# pm25DF['Date of Last Change']
anotherDF['dates']

In [ ]:
print(anotherDF['dates'].dtype)
print(airDF['Date of Last Change'].dtype)
print(anotherDF['fips'].dtype)
print(airDF['fips'].dtype)

Put it all together

In [ ]:
newDF = anotherDF.merge(airDF, left_on=['dates','fips'], right_on=['Date of Last Change','fips'], how='inner')

In [ ]:
# newDF.to_csv('allTogetherNow.gz',index=False,compression='gzip')
outfile = open('allTogetherNow.pkl','wb')
pickle.dump(newDF,outfile)
outfile.close()

Import back in

In [6]:
def importPbz2( file ):
    data = bz2.BZ2File(file,'rb')
    # newDF = pd.read_pickle('allTogetherNow.pkl')
    return pd.read_pickle(data)

newDF = importPbz2('allTogetherNow.pbz2')

And explore...

In [ ]:
print(newDF.shape)
mask = newDF.isnull().any(axis=0)
noNullDF = newDF.loc[:,~mask]
print(noNullDF.shape)

In [21]:
newDF['density']=newDF['latestTotalPopulation']/newDF['LND110210']

In [22]:
# a = [x for x in newDF if x.startswith('ozone')]
b = [x for x in newDF if (x.startswith('JHU')|x.startswith('NYT'))&(~x.endswith('missing'))&('Confirmed' in x)]
# b.extend(a)
b
no2DF = newDF.loc[newDF['pollutant']=='no2',b+['Observation Count','Observation Percent','Arithmetic Mean',\
                                                           '1st Max Value','Mean ugm3','fips','dates','density']]
# ozoneDF.dropna(subset=a,inplace=True)


In [29]:
# ozoneDF.dropna(subset=['Mean ugm3'],inplace=True)
no2fipsDF = no2DF[(no2DF['dates']>='2020-03-11')&(no2DF['dates']<='2021-03-11')].groupby(by=['fips']).agg({'JHU_ConfirmedDeaths.data': 'max', 'NYT_ConfirmedDeaths.data': 'max',\
                              'Mean ugm3': 'mean','density': 'max'})

In [30]:
no2fipsDF.corr()

,JHU_ConfirmedDeaths.data,NYT_ConfirmedDeaths.data,Mean ugm3,density
JHU_ConfirmedDeaths.data,1.000000,0.999807,0.469625,0.449120
NYT_ConfirmedDeaths.data,0.999807,1.000000,0.469777,0.448001
Mean ugm3,0.469625,0.469777,1.000000,0.580541
density,0.449120,0.448001,0.580541,1.000000


In [26]:
def covidPollutantFipsCorr(df, pollutant, cols=[]):
    if cols:
        polDF =  df.loc[df['pollutant']==pollutant,cols+['fips']]
    else:
        polDF = df.loc[df['pollutant']==pollutant,:]
    fipsDF  = polDF.groupby(by='fips').agg('mean')
    return fipsDF.corr()

In [31]:
dateFilterDF = newDF[(newDF['dates']>='2020-03-11')|(newDF['dates']<='2021-03-11')]

NO2

In [42]:
covidPollutantFipsCorr(dateFilterDF, 'no2', b[:5]+['Arithmetic Mean','density'])

,JHU_ConfirmedCases.data,NYT_ConfirmedCases.data,JHU_ConfirmedDeaths.data,JHU_ConfirmedRecoveries.data,NYT_ConfirmedDeaths.data,Arithmetic Mean,density
JHU_ConfirmedCases.data,1.000000,0.999370,0.934337,NaN,0.933333,0.435601,0.347623
NYT_ConfirmedCases.data,0.999370,1.000000,0.933436,NaN,0.932854,0.438036,0.346395
JHU_ConfirmedDeaths.data,0.934337,0.933436,1.000000,NaN,0.999769,0.472795,0.434318
JHU_ConfirmedRecoveries.data,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NYT_ConfirmedDeaths.data,0.933333,0.932854,0.999769,NaN,1.000000,0.471129,0.432409
Arithmetic Mean,0.435601,0.438036,0.472795,NaN,0.471129,1.000000,0.532606
density,0.347623,0.346395,0.434318,NaN,0.432409,0.532606,1.000000


Lead

In [43]:
covidPollutantFipsCorr(dateFilterDF, 'lead', b[:5]+['Arithmetic Mean','density'])

,JHU_ConfirmedCases.data,NYT_ConfirmedCases.data,JHU_ConfirmedDeaths.data,JHU_ConfirmedRecoveries.data,NYT_ConfirmedDeaths.data,Arithmetic Mean,density
JHU_ConfirmedCases.data,1.000000,0.999995,0.986252,NaN,0.987367,-0.078670,0.415553
NYT_ConfirmedCases.data,0.999995,1.000000,0.986216,NaN,0.987334,-0.078643,0.415348
JHU_ConfirmedDeaths.data,0.986252,0.986216,1.000000,NaN,0.999815,-0.076989,0.465977
JHU_ConfirmedRecoveries.data,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NYT_ConfirmedDeaths.data,0.987367,0.987334,0.999815,NaN,1.000000,-0.074662,0.459923
Arithmetic Mean,-0.078670,-0.078643,-0.076989,NaN,-0.074662,1.000000,-0.227619
density,0.415553,0.415348,0.465977,NaN,0.459923,-0.227619,1.000000


Ozone

In [44]:
covidPollutantFipsCorr(dateFilterDF, 'ozone', b[:5]+['Arithmetic Mean','density'])

,JHU_ConfirmedCases.data,NYT_ConfirmedCases.data,JHU_ConfirmedDeaths.data,JHU_ConfirmedRecoveries.data,NYT_ConfirmedDeaths.data,Arithmetic Mean,density
JHU_ConfirmedCases.data,1.000000,0.999442,0.936091,NaN,0.935396,-0.115189,0.456450
NYT_ConfirmedCases.data,0.999442,1.000000,0.935365,NaN,0.935056,-0.112181,0.456486
JHU_ConfirmedDeaths.data,0.936091,0.935365,1.000000,NaN,0.999585,-0.123963,0.495844
JHU_ConfirmedRecoveries.data,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NYT_ConfirmedDeaths.data,0.935396,0.935056,0.999585,NaN,1.000000,-0.121489,0.494708
Arithmetic Mean,-0.115189,-0.112181,-0.123963,NaN,-0.121489,1.000000,-0.168680
density,0.456450,0.456486,0.495844,NaN,0.494708,-0.168680,1.000000


PM2.5

In [45]:
covidPollutantFipsCorr(dateFilterDF, 'pm25', b[:5]+['Arithmetic Mean','density'])

,JHU_ConfirmedCases.data,NYT_ConfirmedCases.data,JHU_ConfirmedDeaths.data,JHU_ConfirmedRecoveries.data,NYT_ConfirmedDeaths.data,Arithmetic Mean,density
JHU_ConfirmedCases.data,1.000000,0.998959,0.918150,NaN,0.915564,-0.086795,0.411385
NYT_ConfirmedCases.data,0.998959,1.000000,0.916921,NaN,0.915038,-0.086633,0.410773
JHU_ConfirmedDeaths.data,0.918150,0.916921,1.000000,NaN,0.999400,-0.085107,0.510096
JHU_ConfirmedRecoveries.data,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NYT_ConfirmedDeaths.data,0.915564,0.915038,0.999400,NaN,1.000000,-0.086228,0.507456
Arithmetic Mean,-0.086795,-0.086633,-0.085107,NaN,-0.086228,1.000000,0.020543
density,0.411385,0.410773,0.510096,NaN,0.507456,0.020543,1.000000


In [80]:
a = no2fipsDF['Arithmetic Mean'].quantile([0,0.25,0.5,0.75,1])
b = no2fipsDF['JHU_ConfirmedDeaths.data'].quantile([0,0.25,0.5,0.75,1])

In [82]:
print(a)
print(b)

0.00     0.614572
0.25     2.990254
0.50     5.266662
0.75     8.579565
1.00    19.021019
Name: Arithmetic Mean, dtype: float64
0.00        0.0
0.25       91.0
0.50      343.0
0.75     1001.0
1.00    10020.0
Name: JHU_ConfirmedDeaths.data, dtype: float64


In [ ]:
corrs =  newDF.corr()

In [4]:
numbDF = newDF.loc[:,newDF.dtypes!='object']
corrs = numbDF.corr()

In [46]:
# # newDF.to_csv('allTogetherNow.gz',index=False,compression='gzip')
# with bz2.BZ2File('allTogetherNow.pbz2','wb') as f:
#     pickle.dump(newDF,f)